In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_clean_and_preprocess').getOrCreate()

In [ ]:
df = spark.read.option("sep","\t").csv("column_headers.tsv",header='true', inferSchema='true')
df1 = spark.read.option("sep","\t").csv("hit_data.tsv",header='false', inferSchema='true')

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import functions as sf
sc = SparkContext.getOrCreate()
sqlc = pyspark.SQLContext(sc)

In [ ]:
result = df.union(df1)

In [ ]:
NoValuedExcludeHit=result.filter(result.exclude_hit == 0)

In [ ]:
ValidHitSourceWithout5=NoValuedExcludeHit.filter(NoValuedExcludeHit.hit_source != 5 )
ValidHitSourceWithout7=ValidHitSourceWithout5.filter(ValidHitSourceWithout5.hit_source != 7 )
ValidHitSourceWithout9=ValidHitSourceWithout7.filter(ValidHitSourceWithout7.hit_source != 9 )
ValidHitSourceWithout8=ValidHitSourceWithout9.filter(ValidHitSourceWithout9.hit_source != 8 )

In [ ]:
MergedColumns = ValidHitSourceWithout8.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#apart from post_ columns considering column obtained after joining the two post_ columns as mentioned above and mcvisid
MergedColumns=MergedColumns.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
MergedColumns=MergedColumns.withColumnRenamed('mcvisid', 'post_mcvisid')

In [ ]:
#dropping the extraneous columns
DropPostVHL=MergedColumns.drop('post_visid_high','post_visid_low')
#len(DropPostVHL.columns)

In [ ]:
UniqueId=DropPostVHL.select('post_uniqueId')
#UniqueId.count()

In [ ]:
#considering only post_ columns
newDataF= DropPostVHL.select(*filter(lambda col:'post_' in col, DropPostVHL.columns))

# Grouping Users who made a Purchase

In [ ]:
#Grouping By userid, events and referrer
Dataframe=newDataF.groupBy("post_uniqueid","post_event_list","post_referrer").count()

In [ ]:
#sorting based on unique userid
Ordered_ByID=Dataframe.orderBy(["post_uniqueid"],ascending=1)

In [ ]:
#drop the null values from event_list
WithoutNaEvent=Ordered_ByID.na.drop(subset=["post_event_list"])

In [ ]:
#Extracting event list containing 1's in between,at end and at the beginning
SelectedRows=WithoutNaEvent.filter(WithoutNaEvent.post_event_list.rlike(',1,') | WithoutNaEvent.post_event_list.endswith(',1') | WithoutNaEvent.post_event_list.startswith('1,') ).select("post_event_list")
#SelectedRows.count()
#SelectedRows.show(10)

In [ ]:
#saving the unique ids for the event_list containing 1's
listids = [list(x.asDict().values())[0] for x in WithoutNaEvent.filter(WithoutNaEvent.post_event_list.rlike(',1,') | WithoutNaEvent.post_event_list.endswith(',1') | WithoutNaEvent.post_event_list.startswith('1,')).select("post_uniqueid").distinct().collect()]

In [ ]:
#count of userids containing 1's
print("{}".format(len(listids)))

In [ ]:
#extracting the entire set of event_lists for the userids in the list containing 1's
for x in listids[0:3]:
    WithoutNaEvent.where(WithoutNaEvent.post_uniqueid == x).show(2)

In [ ]:
from pyspark.sql.functions import col

DataF=WithoutNaEvent.where(col("post_uniqueid").isin(listids))

In [ ]:
DataF.show()

# Grouping Users who Abandoned cart

In [ ]:
#storing all existing distinct users
allids = [list(x.asDict().values())[0] for x in Ordered_ByID.select("post_uniqueid").distinct().collect()]

In [ ]:
print("{}".format(len(allids)))

In [ ]:
#saving the unique ids for the event_list containing 1's
listids = [list(x.asDict().values())[0] for x in WithoutNaEvent.filter(WithoutNaEvent.post_event_list.rlike(',1,') | WithoutNaEvent.post_event_list.endswith(',1') | WithoutNaEvent.post_event_list.startswith('1,')).select("post_uniqueid").distinct().collect()]

In [ ]:
print("{}".format(len(listids)))

In [ ]:
#user list of users who abandoned the cart
negativelistids=list(set(allids) - set(listids))

In [ ]:
print("{}".format(len(negativelistids)))

In [ ]:
#extracting the entire set of event_lists for users who abandoned the carts
for x in negativelistids[0:3]:
    Ordered_ByID.where(Ordered_ByID.post_uniqueid == x).show(5,False)